4.1 소벨 에지 검출

In [1]:
import cv2

img = cv2.imread('dark.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

gradx = cv2.Sobel(gray,cv2.CV_32F,1,0,ksize=3)
grady = cv2.Sobel(gray,cv2.CV_32F,0,1,ksize=3)

sobelx = cv2.convertScaleAbs(gradx)
sobely = cv2.convertScaleAbs(grady)

edge_strength = cv2.addWeighted(sobelx,0.5,sobely,0.5,0)

cv2.imshow('original',gray)
cv2.imshow('sobelx',sobelx)
cv2.imshow('sobely',sobely)
cv2.imshow('edge_strength',edge_strength)

cv2.waitKey()
cv2.destroyAllWindows()

4.2 캐니 에지

In [2]:
import cv2

img = cv2.imread('soccer2.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

canny1 = cv2.Canny(gray,50,150)
canny2 = cv2.Canny(gray,100,200)

cv2.imshow('original',gray)
cv2.imshow('Canny1',canny1)
cv2.imshow('Canny2',canny2)

cv2.waitKey()
cv2.destroyAllWindows()

4.3 에지 맵에서 경계선 찾기

In [3]:
import cv2
import numpy as np

img = cv2.imread('soccer2.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(gray,100,200)

contour, hierarchy = cv2.findContours(canny,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)

lcontour = []

for i in range(len(contour)):
    if contour[i].shape[0] > 100:
        lcontour.append(contour[i])

cv2.drawContours(img,lcontour,-1,(0,255,0),3)

cv2.imshow('original',img)
cv2.imshow('canny',canny)

cv2.waitKey()
cv2.destroyAllWindows()

4.3 허프 변환으로 사과 검출하기

In [4]:
import cv2

img = cv2.imread('apple.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

apples = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,1,200, param1 = 150, param2 = 20, minRadius = 50, maxRadius = 120)
for i in apples[0]:
    cv2.circle(img,(int(i[0]),int(i[1])),int(i[2]),(255,0,0),2)

cv2.imshow('apple detection',img)

cv2.waitKey()
cv2.destroyAllWindows()

4.4 SLIC 알고리즘으로 입력 영상을 슈퍼 화소 분할하기

In [5]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [16]:
import cv2
import skimage
import numpy as np
img = skimage.data.coffee()

cv2.imshow('coffee', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

slic1 = skimage.segmentation.slic(img, compactness = 20, n_segments = 600)
sp_img = skimage.segmentation.mark_boundaries(img, slic1)
sp_img = np.uint8(sp_img * 255.0)

cv2.imshow('super pixel', cv2.cvtColor(sp_img,cv2.COLOR_RGB2BGR))

cv2.waitKey(0)
cv2.destroyAllWindows()

4.4 정규화 절단 알고리즘으로 영역 분할하기

In [1]:
import cv2
import skimage
import numpy as np

coffee = skimage.data.coffee()

slic = skimage.segmentation.slic(coffee, compactness = 20, n_segments = 600)

g = skimage.future.graph.rag_mean_color(coffee, slic, mode = 'similarity')

ncut = skimage.future.graph.cut_normalized(slic, g)

marking = skimage.segmentation.mark_boundaries(coffee, ncut)
ncut_coffee = np.uint8(marking * 255.0)

cv2.imshow('normalized cut', cv2.cvtColor(ncut_coffee, cv2.COLOR_RGB2BGR))
cv2.waitKey()
cv2.destroyAllWindows()

AttributeError: No skimage.future attribute graph

4.5 GrabCut으로 물체 분할

In [5]:
import cv2
import numpy as np


img = cv2.imread('soccer2.jpg')
img_show = np.copy(img)

mask = np.zeros((img.shape[0], img.shape[1]), np.uint8)
mask[:,:] = cv2.GC_PR_BGD

brushsiz = 9
lcolor, rcolor = (255,0,0), (0,0,255)

def painting(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img_show, (x,y), brushsiz, lcolor, -1)
        cv2.circle(mask, (x,y), brushsiz, cv2.GC_FGD, -1)

    elif event == cv2.EVENT_RBUTTONDOWN:
        cv2.circle(img_show, (x,y), brushsiz, rcolor, -1)
        cv2.circle(mask, (x,y), brushsiz, cv2.GC_BGD, -1)

    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        cv2.circle(img_show, (x,y), brushsiz, lcolor, -1)
        cv2.circle(mask, (x,y), brushsiz, cv2.GC_FGD, -1)

    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_RBUTTON:
        cv2.circle(img_show, (x,y), brushsiz, rcolor, -1)
        cv2.circle(mask, (x,y), brushsiz, cv2.GC_BGD, -1)
        
    cv2.imshow('painting', img_show)

cv2.namedWindow('painting')
cv2.setMouseCallback('painting',painting)


while True:
    if cv2.waitKey(1) == ord('q'):
        break

background = np.zeros((1,65), np.float64)
foreground = np.zeros((1,65), np.float64)

cv2.grabCut(img, mask, None, background, foreground, 5, cv2.GC_INIT_WITH_MASK)
mask2 = np.where((mask == cv2.GC_BGD)|(mask == cv2.GC_PR_BGD), 0,1).astype('uint8')
grab = img * mask2[:,:,np.newaxis]
cv2.imshow('grab cut image', grab)

cv2.waitKey()
cv2.destroyAllWindows()

4.6 이진 영역의 특징을 추출하는 함수 사용

In [7]:
import skimage
import numpy as np
import cv2

orig = skimage.data.horse()
img = 255 - np.uint8(orig) * 255
cv2.imshow('horse', img)

contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

img2 = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
cv2. drawContours(img2, contours, -1, (255,0,255), 2)
cv2.imshow('horse with contour', img2)

contour = contours[0]

m = cv2.moments(contour)
area = cv2.contourArea(contour)
cx, cy = m['m10']/m['m00'], m['m01']/m['m00']
perimeter = cv2.arcLength(contour, True)
roundness = (4.0 * np.pi * area)/(perimeter * perimeter)
print(f'면적 = {area}\n중점 = ({cx}, {cy})\n둘레 = {perimeter}\n둥근 정도 = {roundness}')

img3 = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

contour_approx = cv2.approxPolyDP(contour, 8, True)
cv2.drawContours(img3, [contour_approx],-1, (0,255,0),2)

hull = cv2.convexHull(contour)
hull = hull.reshape(1, hull.shape[0], hull.shape[2])
cv2.drawContours(img3,hull, -1, (0,0,255),2)

cv2.imshow('horse with line segments and convex hull', img3)

cv2.waitKey()
cv2.destroyAllWindows()

면적 = 42390.0
중점 = (187.72464024534088, 144.43640402610677)
둘레 = 2296.7291333675385
둥근 정도 = 0.1009842680321435
